In [1]:
#Using google colab for getting the dataset from google drive
# !unzip /content/drive/MyDrive/dog_cat_dataset.zip -d /content/

In [2]:
#Accessing the google drive from colab
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#importing libraries
from tensorflow import keras
import matplotlib.pyplot as plt

In [5]:
#Preprocessing and loading the dataset
#I have not done extra preprocessing
train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'training_set/training_set/',
        target_size=(227, 227),
        batch_size=64,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'test_set/test_set/',
        target_size=(227, 227),
        batch_size=64,
        class_mode='categorical')

#Used pre-trained Inception model
#I tried alexnet and efficientnet but I didn't get the accuracy better than Inception model
#resized the image as per the model requirement
model = keras.applications.InceptionV3(
    weights=None, input_shape=(227, 227, 3), classes=2)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [8]:
#Callbacks is used for earlystopping and for adding some functionality to train a model
#ReduceLROnPlateau is used for reducing the learning rate from 0.001 till it satisfy the minimum learning rate till the condition
#Adam is the best optimizer that's why I have used
#I used batch_size 256 because I tried different different batch size but I found this performing better
#0ne more advange of this batch_size, it reduces the some what training time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto', restore_best_weights=True)
rlrop = ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, factor=0.5, min_lr=1e-6, verbose=1)
from keras.optimizers import Adam
callbacks = [earlystopping, rlrop]
model.compile(optimizer=Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_generator,batch_size=256, epochs=200, validation_data=validation_generator,callbacks=callbacks)

Epoch 1/200
126/126 [==============================] - 167s 1s/step - loss: 0.6529 - accuracy: 0.6418 - val_loss: 0.8426 - val_accuracy: 0.4998 - lr: 0.0010
Epoch 2/200
126/126 [==============================] - 145s 1s/step - loss: 0.6388 - accuracy: 0.6337 - val_loss: 0.8412 - val_accuracy: 0.4998 - lr: 0.0010
Epoch 3/200
126/126 [==============================] - 148s 1s/step - loss: 0.5536 - accuracy: 0.7245 - val_loss: 0.8764 - val_accuracy: 0.5818 - lr: 0.0010
Epoch 4/200
126/126 [==============================] - 154s 1s/step - loss: 0.4847 - accuracy: 0.7744 - val_loss: 1.3323 - val_accuracy: 0.5843 - lr: 0.0010
Epoch 5/200
126/126 [==============================] - 150s 1s/step - loss: 0.4139 - accuracy: 0.8130 - val_loss: 0.5984 - val_accuracy: 0.7237 - lr: 0.0010
Epoch 6/200
126/126 [==============================] - 150s 1s/step - loss: 0.3544 - accuracy: 0.8437 - val_loss: 0.4750 - val_accuracy: 0.8216 - lr: 0.0010
Epoch 7/200
126/126 [==============================] - 146

In [ ]:
#I used here earlystopping which is giving the best accuracy on test data 21st epoch
#Test accuracy or validation accuracy is 93.23% and it is the best accuracy from all the models I have tried
#Training accuracy is 94.10%
